## Setup

In [1]:
%run ../../utils.ipynb
# %run objects.ipynb

import time
import pickle

import gpflow
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, roc_auc_score, mean_absolute_error
from sklearn.preprocessing import LabelBinarizer, scale, robust_scale, normalize

from ordinal_likelihood import Ordinal

W0903 11:21:21.502460 4534601152 deprecation_wrapper.py:119] From /Users/rob/.pyenv/versions/brookfield/lib/python3.5/site-packages/gpflow/session_manager.py:31: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0903 11:21:21.513328 4534601152 deprecation_wrapper.py:119] From /Users/rob/.pyenv/versions/brookfield/lib/python3.5/site-packages/gpflow/misc.py:27: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0903 11:21:21.617810 4534601152 deprecation_wrapper.py:119] From /Users/rob/.pyenv/versions/brookfield/lib/python3.5/site-packages/gpflow/training/tensorflow_optimizer.py:169: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W0903 11:21:21.619091 4534601152 deprecation_wrapper.py:119] From /Users/rob/.pyenv/versions/brookfield/lib/python3.5/site-packages/gpflow/training/tensorflow_optimizer.py:156: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1

## Functions

In [2]:
def rec_dd():
        return defaultdict(rec_dd)

In [3]:
## Moved
def get_kern(kern, dims, shape):
    kern = kern.split('_')
    with gpflow.defer_build():
        if kern[0] == 'Matern12': kernel = gpflow.kernels.Matern12(input_dim=dims)
        if kern[0] == 'Matern32': kernel = gpflow.kernels.Matern32(input_dim=dims) 
        if kern[0] == 'Matern52': kernel = gpflow.kernels.Matern52(input_dim=dims) 
        if kern[0] == 'RBF':      kernel = gpflow.kernels.RBF(input_dim=dims) 
        kernel.variance.prior = gpflow.priors.Gamma(scale=1,shape=shape)
        if len(kern) == 2 and kern[1] == 'Linear':
            linear = gpflow.kernels.Linear(input_dim=dims)
            linear.variance.prior = gpflow.priors.Gamma(scale=1,shape=shape)
            kernel += linear
    return kernel

In [4]:
## Moved
# create x and y arrays
def x_and_y(x, y):
    change = {'decrease':     2,
              'constant':     1, 
              'increase':     0,
              'fewer':        2,
              'same':         1,
              'more':         0,
              'not_increase': 1}
    x = np.array(x)
    y = np.array(pd.Series(y).replace(change).values.astype('int64')).reshape(y.shape[0], 1)
    return x, y

In [5]:
## Moved
# get bin edges and likelihood
def create_likelihood(y):
    bin_edges = np.array(np.arange(np.unique(y).size + 1), dtype=float)
    # Need to check in on this, tutorial does the below which ends up with negative bins
    # bin_edges = bin_edges - bin_edges.mean()
    bin_edges = bin_edges - .5
    return Ordinal(bin_edges)

In [6]:
## Moved
# build a model with this likelihood
def build_model(x_train, y_train, likelihood, kernel):
    gaussian_model = gpflow.models.VGP(tf.cast(x_train, tf.float64),
                                       y_train, 
                                       kern=kernel,
                                       likelihood=likelihood)
    # fit the model
    gpflow.train.ScipyOptimizer().minimize(gaussian_model)
    return gaussian_model

In [7]:
## Moved
def predictive_density(model, x):
    ys = np.arange(np.max(model.Y.value+1)).reshape([-1, 1])
    x_new_vec = x*np.ones_like(ys)
    # for predict_density x and y need to have the same number of rows
    dens = np.exp(model.predict_density(x_new_vec, ys))
    return dens

In [8]:
## Moved
# get predictive densities
def test_model(model, x_test):
    densities = []
    # Predictive density for a single input x
    for x in x_test:
        densities.append(predictive_density(model, x))
    return densities

In [9]:
## Not used anymore
# get accuracy
def accuracy(y_test, densities):
    score = 0
    for index, y in enumerate(y_test):
        if y == np.argmax(densities[index]): score += 1
    return score/len(y_test)

In [10]:
## Moved
# multiclass ROC
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [11]:
# figure out the mse
## Moved
def scale_pred(pred):
    factor = 1/sum(pred)
    scaled = []
    for p in pred:
        scaled.append(p * factor)
    return tuple(scaled)

## Abstracted out, assuming true y_test_dist is passed in
def calc_errors(x_test_all, densities, agg_level, dist):
    mapping = {0: 'increase',
               1: 'constant/not increase',
               2: 'decrease'}
    y_true = []
    y_pred = []
    true_class = []
    pred_class = []
    # true then pred
    confusion = defaultdict(lambda: defaultdict(int))
    for index, x in enumerate(x_test_all):
        pred = tuple([p for [p] in densities[index]])
        pred = scale_pred(pred)
        x = tuple(x)
        true_info = noc_dict[dist][x]
        noc = true_info['noc']
        if len(pred) == 3: true_dist = true_info['share']
        else:              true_dist = true_info['share_binned']
        y_true.append(true_dist)
        y_pred.append(pred)
        y_t = np.argmax(true_dist)
        y_p = np.argmax(pred)
        true_class.append(y_t)
        pred_class.append(y_p)
        confusion[mapping[y_t]][mapping[y_p]] += 1
        
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    roc = multiclass_roc_auc_score(true_class, pred_class)
    return mse, mae, roc, confusion

In [12]:
# do cross validation
def cv(x, y, x_all, y_all, kern, agg_level, dist, group):
    if agg_level == 'ind': x_all = x_all.drop(['noc', 'participant.id'], axis=1)
    x_all = np.asarray(x_all)
    dims = len(x[0])
    kf = GroupKFold(n_splits=5)
    scores = []
#     results = []
    mse = []
    mae = []
    roc = []
    total_confusion = defaultdict(lambda: defaultdict(int))

    for train_index, test_index in kf.split(x, groups=group):
        start = time.time()
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        x_test_all = x_all[test_index]
        unique, unique_indices = np.unique(x_test, axis=0, return_index=True)        
        kernel = get_kern(kern, dims, len(np.unique(y)))
        likelihood = create_likelihood(y_all)
        model = build_model(x_train, y_train, likelihood, kernel)
        densities = test_model(model, [x_test[i] for i in unique_indices])
        scores.append(accuracy([y_test[i] for i in unique_indices], densities))
        mse_error, mae_error, roc_error, confusion = calc_errors([x_test_all[i] for i in unique_indices], 
                                                                 densities, 
                                                                 agg_level, 
                                                                 dist)
        mse.append(mse_error)
        mae.append(mae_error)
        roc.append(roc_error)
        for y_t in confusion:
            for y_p in confusion[y_t]:
                total_confusion[y_t][y_p] += confusion[y_t][y_p]
#         results.append(create_results([x_test[i] for i in unique_indices], densities))
        end = time.time()
        print('Fold tested in (sec):', end - start)
        del kernel, likelihood, model

    return {'Avg. score':  sum(scores)/len(scores),
            'Avg. mse':    sum(mse)/len(mse),
            'Avg. mae':    sum(mae)/len(mae),
            'Avg. roc':    sum(roc)/len(roc),
            'Confusion':   dict(total_confusion),
            'All scores':  scores,
            'All mse':     mse,
            'All mae':     mae,
            'All roc':     roc}
#             'All results': results}

In [13]:
def tester(model, save_path):
    # results will be saved under gp_results/{save_path}
    def run_test(scaler, normal):
        start = time.time()
        x, y = x_and_y(cleaned_x, y_all)
        scale_name = params
        if scaler:
            scale_name += '_' + scaler.__name__
            x = scaler(x)
        if normal: 
            x = normalize(x)
            scale_name += '_' + normalize.__name__
        print('Testing:', scale_name)
        result = cv(x, y, x_all, y_all, kern, agg_level, dist, group)
        end = time.time()
        print('Accuracy:', result['Avg. score'])
        print('MSE:', result['Avg. mse'])
        print('MAE:', result['Avg. mae'])
        print('ROC:', result['Avg. roc'])
        print('Confusion:', result['Confusion'])
        print('Time elapsed (sec):', end - start)
        pickle_results(result, scale_name)
        print('Results saved to disk')
        
    params = '/'.join(['full_model', 'combined_kernel', name])
    scales = [robust_scale, scale]
    if agg_level == 'agg': 
        group = agg_nocs
    else:
        cleaned_x = x_all.drop(['noc', 'participant.id'], axis=1)
        group = x_all['noc']
#     for scaler in scales:
#         for normal in range(2):
#             run_test(scaler, normal)
    run_test(scale, 1)

In [14]:
def pickle_results(results, params):
    with open('../gp_results/{}.pkl'.format(params), 'wb') as f:
        pickle.dump(results, f)